In [1]:
import numpy as np 

sample = np.loadtxt("../matrix/6_random_adj_mat_9.txt")
sample

array([[0.        , 0.56182928, 0.44047985, 0.32644743, 0.71033334,
        0.33938534],
       [0.56182928, 0.        , 0.87221518, 0.69626131, 1.15341181,
        0.40357252],
       [0.44047985, 0.87221518, 0.        , 0.17749686, 0.28450999,
        0.482133  ],
       [0.32644743, 0.69626131, 0.17749686, 0.        , 0.4573035 ,
        0.30488284],
       [0.71033334, 1.15341181, 0.28450999, 0.4573035 , 0.        ,
        0.75659622],
       [0.33938534, 0.40357252, 0.482133  , 0.30488284, 0.75659622,
        0.        ]])

In [2]:
#Using nearest-neighbor as a "random" solution

def randomp(x):
    while True:
        path = [i for i in range(len(x))]
        np.random.shuffle(path)
        path.append(path[0])
        cost = 0

        cont = True

        for idx in range(0, len(path) - 1):
            edge = x[path[idx]][path[idx+1]]
            if edge == 0:
                cont = False
                break
            else:
                cost += edge

        if cont:
            return path, float(cost)
        
randomp(sample)

([4, 3, 5, 0, 1, 2, 4], 2.8201261201176187)

In [3]:
#Helper to find n random valid swaps (assuming x wraps around itself)
def find_valid_swap(x, path, cost, swaps):

    tested = set()

    while len(swaps) > 0:
        new_cost = cost

        (i, j) = np.random.choice(list(swaps))
        swaps.remove((i, j))
        if (i, j) in tested:
            continue

        idx_to_sub = { (i, i+1), (i, i-1), (j, j+1), (j, j-1) }
        idx_to_add = { (i, j+1), (i, j-1), (j, i-1), (j, i+1) }
        if i == 0:
            idx_to_sub.remove((i, i-1))
            idx_to_add.remove((j, i-1))

        tobreak = False
        for a, b in idx_to_add:
            if x[path[a]][path[b]] == 0:
                tobreak = True
                break
            new_cost += x[path[a]][path[b]]

        if tobreak:
            continue
        
        for a, b in idx_to_sub:
            new_cost -= x[path[a]][path[b]]
        
        newpth = path.copy()
        tmp = newpth[i]; newpth[i] = newpth[j]; newpth[j] = tmp
        newpth[-1] = newpth[0] #In case swap since j != end

        return i, j, newpth, new_cost
    
    return None


    #A b c d E A


In [4]:


def hill_climbing(x, num_restarts, find_valid_swap = find_valid_swap, n_neighbors = 10):
    min_path, min_cost = [], float('inf')

    for _ in range(num_restarts):
        local_path, local_cost = randomp(x)
        base_path = local_path.copy()

        while True:

            improved = False

            #Attempt on n neighbors
            neighbors_left = n_neighbors
            swaps = { (0, 0) }

            while neighbors_left > 0:
                #Generate random + check valid + recalc cost
                i, j = 0, 0
                p, c = [], float('inf')
                while (i, j) in swaps:
                    i, j, p, c = find_valid_swap(x, local_path, local_cost)

                #Update
                if c < local_cost:
                    local_path, local_cost = p, c
                    improved = True
            
                swaps.add(tuple(sorted((i, j))))
                neighbors_left -= 1

            #If no progress has been made
            if not improved:
                break


        #Using the path we found
        if local_cost < min_cost:
            min_path, min_cost = local_path, local_cost


        # a, B, C, D, e, A



    return min_path, float(min_cost)


p, c = hill_climbing(sample, 2, find_valid_swap, 4)
p, c

TypeError: find_valid_swap() missing 1 required positional argument: 'swaps'

In [ ]:
def fact(x):
    if x == 0:
        return 1
    return x * fact(x-1)

In [ ]:
#Simulated Annealing
def simulated_annealing(x, max_iterations, alpha, initial_temperature, find_valid_swap = find_valid_swap, n_neighbors = 10):

    #To avoid nneighbors recursive depth
    n_neighbors = min(n_neighbors, fact(x) / ( fact(2) * fact(x-2) ))
    print(f"NN={n_neighbors}")

    #Only running on one state, but attempting to climb a hill
    local_path, local_cost = randomp(x)
    tmp = initial_temperature

    cpg = []
    ppg = []


    for iteridx in range(max_iterations):


        #Finding the best neighbor
        neighbors_left = n_neighbors
        swaps = { (0, 0) }

        best_path, best_cost = local_path.copy(), local_cost

        neighbors_left = n_neighbors
        while neighbors_left > 0 and len(swaps) < n_neighbors:
            #Generate random + check valid + recalc cost
            i, j = 0, 0
            p, c = [], float('inf')
            while (i, j) in swaps:
                i, j, p, c = find_valid_swap(x, local_path, local_cost)
                print(i, j)
            print(swaps)
            #Update
            if c < local_cost:
                local_path, local_cost = p, c
                improved = True
            
                swaps.add(tuple(sorted((i, j))))
                neighbors_left -= 1

            #Accepting this solution if formula sat
            #Ignoring warnings for prod (solve err in nb)
            else:
                with np.errstate(over = "ignore"):
                    prob = np.exp( (c - best_cost) / tmp )
                if prob > 1 or np.random.choice([True, False], p=[prob, 1-prob]):
                    best_path, best_cost = p, c
                    tmp *= alpha
                
                swaps.add(tuple(sorted((i, j))))
                neighbors_left -= 1

        #Update if we made progress
        if best_cost <= local_cost:
            local_path, local_cost = best_path, best_cost

        ppg.append((iteridx, best_path))
        cpg.append((iteridx, best_cost))


    return local_path, float(local_cost), cpg


In [ ]:

sample = np.loadtxt("../matrix/5_random_adj_mat_4.txt")
p, c, _ = simulated_annealing(sample, 5, 0.1, 10, find_valid_swap)
p, c, _

NN=-0.0


([4, 2, 1, 3, 0, 4],
 2.155601984032107,
 [(0, 2.155601984032107),
  (1, 2.155601984032107),
  (2, 2.155601984032107),
  (3, 2.155601984032107),
  (4, 2.155601984032107)])

In [8]:
#Helper to find n random valid swaps (assuming x wraps around itself) or returns None is none are available
def find_valid_swap(x, path, cost, swaps):

    tested = set()
    rdepth = 0 #Max 100 as guard, should never happen and will throw errors in method if it does as a form of alert

    while rdepth < 100:
        new_cost = cost
        
        rdepth += 1
        

        if swaps != None and len(swaps) == 0:
            break
 

        if swaps is not None:
            lswap = list(swaps)
            np.random.shuffle(lswap)
            (i, j) = lswap[0]
            swaps.remove((i, j))

        else:
            i = np.random.randint(0, len(x) - 1)
            j = np.random.randint(i, len(x))


        if (i, j) in tested:
            continue

        idx_to_sub = { (i, i+1), (i, i-1), (j, j+1), (j, j-1) }
        idx_to_add = { (i, j+1), (i, j-1), (j, i-1), (j, i+1) }
        if i == 0:
            idx_to_sub.remove((i, i-1))
            idx_to_add.remove((j, i-1))

        tobreak = False
        for a, b in idx_to_add:
            if x[path[a]][path[b]] == 0:
                tobreak = True
                break
            new_cost += x[path[a]][path[b]]

        if tobreak:
            continue
        
        for a, b in idx_to_sub:
            new_cost -= x[path[a]][path[b]]
        
        newpth = path.copy()
        tmp = newpth[i]; newpth[i] = newpth[j]; newpth[j] = tmp
        newpth[-1] = newpth[0] #In case swap since j != end

        return i, j, newpth, new_cost
    
    return None

    



#MPX crossover for genetic impl
def mpx_crossover(x, p1, p2):
    l = len(p1) // 2
    if len(p1) > 10:
        if len(p1) // 2 > 10:
            l = np.random.randint(10, len(p1) // 2)
        else:
            l = 10

    while True:
        startidx = np.random.randint(0, len(p1) - l)
        taken = set(p1[startidx : startidx + l])

        path = p1[startidx : startidx + l]
        cost = 0
        cont = True
        for idx in range(0, len(path) - 1):
            if x[path[idx]][path[idx+1]] == 0:
                cont = False; break
            cost += x[path[idx]][path[idx+1]]
        
        if cont:
            for el in p2:
                if el not in taken:
                    if x[el][path[-1]] == 0:
                        cont = False; break
                    cost += x[el][path[-1]]
                    path.append(el)
                    taken.add(el)

            if x[path[-1]][path[0]] > 0:
                cost += x[path[-1]][path[0]]
                path.append(path[0])
                return path, float(cost)


In [14]:
def genetic_algo(x, mutation_chance, population_size, num_generations, 
                 crossover=mpx_crossover, find_valid_swap=find_valid_swap, elim_pc = 0.1): 

    paths = sorted([randomp(x) for _ in range(population_size)], key = lambda p : -p[-1])
    cpg = []
    ppg = []

    for genidx in range(num_generations):

        children = []

        while len(paths) > 0:
            if len(paths) == 1:
                p, c = paths.pop()
                if np.random.choice([True, False], p=[mutation_chance, 1-mutation_chance]):
                    _, _, p, c = find_valid_swap(x, p, c, None)
                children.append((p, c))
            else:
                idx = np.random.randint(0, len(paths))
                idy = np.random.randint(0, len(paths))
                while idy == idx:
                    idy = np.random.randint(0, len(paths))

                p1, _ = paths[idx]
                p2, _ = paths[idy]
                crossp, crossc = crossover(x, p1, p2)

                if np.random.choice([True, False], p=[mutation_chance, 1-mutation_chance]):
                    _, _, crossp, crossc = find_valid_swap(x, crossp, crossc, None)

                children.append((crossp, crossc))
                paths.pop(idx)
                if idy < idx:
                    paths.pop(idy)
                else:
                    paths.pop(idy - 1)

        children = sorted(children, key = lambda p : -p[-1])[int(len(children) * elim_pc) : ]
        print(children)
        paths = children
   
        cpg.append((genidx, paths[-1][-1]))
        ppg.append((genidx, paths[-1][0]))
  
    p, c = paths[-1]
    return p, float(c), cpg
    
    
p, c, _ = genetic_algo(sample, 0.3, 10, 3, mpx_crossover, find_valid_swap, 0.1)
p, c

[([3, 1, 4, 0, 2, 5, 3], 3.7875021380391076), ([0, 5, 3, 4, 1, 2, 0], 3.567678503158507), ([3, 1, 0, 4, 5, 2, 3], 3.3846500096098637), ([0, 2, 4, 1, 5, 3, 0], 2.913304434122705), ([4, 2, 1, 3, 0, 5, 4], np.float64(2.858204581256531))]
[([0, 4, 5, 2, 1, 3, 0], 3.8439864700103987), ([3, 1, 4, 0, 2, 5, 3], 3.7875021380391076), ([2, 1, 3, 0, 5, 4, 2], 3.2754154665014843)]
[([1, 4, 0, 5, 2, 3, 1], np.float64(3.5590216544157465)), ([2, 1, 3, 0, 5, 4, 2], 3.2754154665014843)]


([2, 1, 3, 0, 5, 4, 2], 3.2754154665014843)